In [1]:
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from xgboost import plot_importance
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression, f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.utils import shuffle
from sklearn import metrics
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import time
import scipy
from sklearn.svm import SVR
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.multitest import fdrcorrection
from statsmodels.stats import multitest
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.ensemble import ExtraTreesClassifier
from collections import Counter
import heapq
import statsmodels.api as sm
import re
from sklearn.preprocessing import MinMaxScaler
import os
from scipy.stats import linregress

In [2]:
compl_df =  pd.read_excel('Data/complexity.xlsx', index_col=0)
compl_df

,SSV_IFG_L,SSV_IFG_R,SSV_TEMP_left,SSV_TEMP_right,SSV_OCC_left,SSV_OCC_right,WMEAN_ALL,PE_ALL,FD_ALL,Age,Sex,Educ,MMSE,GINI,ODQ
0,0.006332,0.005878,0.002696,0.007597,0.001630,0.006095,0.686534,1.115442,0.858276,33.0,1.0,19.0,30.0,40.70,99.282421
1,0.004393,0.006609,0.005131,0.002860,0.008439,0.004499,0.685793,1.107025,0.770389,57.0,2.0,13.0,NaN,40.70,99.739738
2,0.004991,0.012247,0.004234,0.008351,0.004922,0.004755,0.690579,1.220202,0.928685,32.0,1.0,10.0,NaN,40.70,99.361739
3,0.009637,0.010634,0.002398,0.005989,0.006487,0.008972,0.691175,1.161459,0.868296,21.0,1.0,13.0,NaN,40.70,99.561273
4,0.047744,0.044420,0.021100,0.041289,0.033726,0.039503,0.761280,1.304064,1.329737,19.0,1.0,13.0,30.0,40.70,99.822774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,0.039185,0.057890,0.028184,0.045887,0.023076,0.027280,0.763815,1.323419,1.369073,57.0,2.0,17.0,30.0,44.92,98.650235
1444,0.015117,0.029721,0.009227,0.020449,0.005755,0.006213,0.704867,1.209097,1.106284,48.0,2.0,17.0,28.0,44.92,99.599221
1445,0.020988,0.022166,0.017896,0.015321,0.017710,0.014295,0.712716,1.211582,1.218559,57.0,2.0,17.0,29.0,44.92,99.675799
1446,0.008930,0.008111,0.056805,0.016624,0.108049,0.106354,0.744955,1.506019,1.242872,69.0,2.0,12.0,30.0,44.92,99.641446


In [3]:
def normalize_value(value, max_value):

    return value / max_value if max_value > 0 else 0

def custom_normalize(row, outcome):

    max_val_lt = row['max_value_lt_threshold_' + outcome]
    max_val_gt = row['max_value_gt_threshold_' + outcome]
    
    if row[outcome] < max_val_lt:
        return normalize_value(row[outcome], max_val_lt)
    else:
        return 1- normalize_value(row[outcome], max_val_gt)

In [4]:
outcomes_list = list(compl_df.iloc[:,0:-6].columns)
outcomes_list

['SSV_IFG_L',
 'SSV_IFG_R',
 'SSV_TEMP_left',
 'SSV_TEMP_right',
 'SSV_OCC_left',
 'SSV_OCC_right',
 'WMEAN_ALL',
 'PE_ALL',
 'FD_ALL']

In [5]:
for outcome in outcomes_list:
    mean_outcome = compl_df[outcome].mean()
    std_outcome = compl_df[outcome].std()
    
    compl_df['max_value_lt_threshold_' + outcome] = compl_df[compl_df[outcome] < (mean_outcome - 1.2 * std_outcome)][outcome].max()
    compl_df['max_value_gt_threshold_' + outcome] = compl_df[compl_df[outcome] >= (mean_outcome - 1.2 * std_outcome)][outcome].max()


In [6]:
for outcome in outcomes_list:
    compl_df[outcome + '_normalized'] = compl_df.apply(custom_normalize, axis=1, outcome=outcome)

In [7]:
compl_df

,SSV_IFG_L,SSV_IFG_R,SSV_TEMP_left,SSV_TEMP_right,SSV_OCC_left,SSV_OCC_right,WMEAN_ALL,PE_ALL,FD_ALL,Age,...,max_value_gt_threshold_FD_ALL,SSV_IFG_L_normalized,SSV_IFG_R_normalized,SSV_TEMP_left_normalized,SSV_TEMP_right_normalized,SSV_OCC_left_normalized,SSV_OCC_right_normalized,WMEAN_ALL_normalized,PE_ALL_normalized,FD_ALL_normalized
0,0.006332,0.005878,0.002696,0.007597,0.001630,0.006095,0.686534,1.115442,0.858276,33.0,...,2.012878,0.190413,0.165287,0.096433,0.232730,0.073717,0.259730,0.933220,0.845839,0.711611
1,0.004393,0.006609,0.005131,0.002860,0.008439,0.004499,0.685793,1.107025,0.770389,57.0,...,2.012878,0.132095,0.185825,0.183533,0.087604,0.381551,0.191702,0.932213,0.839456,0.638743
2,0.004991,0.012247,0.004234,0.008351,0.004922,0.004755,0.690579,1.220202,0.928685,32.0,...,2.012878,0.150073,0.344356,0.151444,0.255849,0.222539,0.202619,0.938719,0.925279,0.769988
3,0.009637,0.010634,0.002398,0.005989,0.006487,0.008972,0.691175,1.161459,0.868296,21.0,...,2.012878,0.289784,0.299017,0.085762,0.183465,0.293292,0.382314,0.939529,0.880734,0.719919
4,0.047744,0.044420,0.021100,0.041289,0.033726,0.039503,0.761280,1.304064,1.329737,19.0,...,2.012878,0.807744,0.837941,0.754722,0.841393,0.872932,0.869626,0.505173,0.988871,0.339385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,0.039185,0.057890,0.028184,0.045887,0.023076,0.027280,0.763815,1.323419,1.369073,57.0,...,2.012878,0.842209,0.788797,0.897937,0.823732,0.913060,0.909967,0.503525,0.260651,0.319843
1444,0.015117,0.029721,0.009227,0.020449,0.005755,0.006213,0.704867,1.209097,1.106284,48.0,...,2.012878,0.454584,0.835698,0.330024,0.626481,0.260202,0.264769,0.958140,0.916858,0.917239
1445,0.020988,0.022166,0.017896,0.015321,0.017710,0.014295,0.712716,1.211582,1.218559,57.0,...,2.012878,0.631121,0.623260,0.640119,0.469370,0.800715,0.609143,0.968810,0.918742,0.394619
1446,0.008930,0.008111,0.056805,0.016624,0.108049,0.106354,0.744955,1.506019,1.242872,69.0,...,2.012878,0.268544,0.228080,0.794292,0.509293,0.592913,0.648992,0.515784,0.158638,0.382540


In [8]:
def classifier_ROC(X_, y, fig_show = True, params_ = None):
    
    X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=10000, stratify=y)

    #lista = list(sfs_.k_feature_names_)

    #X_train = X_train[lista]
    #X_test = X_test[lista]

    cv    = RepeatedKFold(n_splits=10, n_repeats=2, random_state=101)
    folds = [(train,test) for train, test in cv.split(X_train, y_train)]
    metrics = ['auc', 'fpr', 'tpr', 'thresholds', 'f1_score', 'accuracy_score', 'recall_score', 'precision_score', 'confusion_matrix', 'model']
    results = {
        'train': {m:[] for m in metrics},
        'val'  : {m:[] for m in metrics},
        'test' : {m:[] for m in metrics}
    }
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)


    if(params_!=None):
        params = params_
    else:
        params = {
            'objective'   : 'binary:logistic',
            'eval_metric' : 'logloss'
        }


    plt.rcParams["figure.figsize"] = (10,10)
    dtest = xgb.DMatrix(X_test, label=y_test)
    for train, test in tqdm(folds, total=len(folds)):
        dtrain = xgb.DMatrix(X_train.iloc[train,:], label=y_train.iloc[train])
        dval   = xgb.DMatrix(X_train.iloc[test,:], label=y_train.iloc[test])
        model  = xgb.train(
            dtrain                = dtrain,
            params                = params, 
            evals                 = [(dtrain, 'train'), (dval, 'val')],
            num_boost_round       = 1000,
            verbose_eval          = False,
            early_stopping_rounds = 10,
        )
        sets = [dtrain, dval, dtest]

        for i,ds in enumerate(results.keys()):
            y_preds              = model.predict(sets[i])
            labels               = sets[i].get_label()
            fpr, tpr, thresholds = roc_curve(labels, y_preds)
            results[ds]['fpr'].append(fpr)
            results[ds]['tpr'].append(tpr)
            results[ds]['thresholds'].append(thresholds)
            results[ds]['auc'].append(roc_auc_score(labels, y_preds))

            results[ds]['f1_score'].append(f1_score(labels, np.round(y_preds)))
            results[ds]['accuracy_score'].append(accuracy_score(labels, np.round(y_preds)))
            results[ds]['recall_score'].append(recall_score(labels, np.round(y_preds)))
            results[ds]['precision_score'].append(precision_score(labels, np.round(y_preds)))
            results[ds]['confusion_matrix'].append(confusion_matrix(labels, np.round(y_preds)))
            results[ds]['model'].append(model)


    kind = 'test'
    c_fill      = 'rgba(52, 152, 219, 0.2)'
    c_line      = 'rgba(52, 152, 219, 0.5)'
    c_line_main = 'rgba(41, 128, 185, 1.0)'
    c_grid      = 'rgba(189, 195, 199, 0.5)'
    c_annot     = 'rgba(149, 165, 166, 0.5)'
    c_highlight = 'rgba(192, 57, 43, 1.0)'

    fpr_mean    = np.linspace(0, 1, 100)
    interp_tprs = []

    for i in range(10*2):
        fpr           = results[kind]['fpr'][i]
        tpr           = results[kind]['tpr'][i]
        interp_tpr    = np.interp(fpr_mean, fpr, tpr)
        interp_tpr[0] = 0.0
        interp_tprs.append(interp_tpr)

    tpr_mean     = np.mean(interp_tprs, axis=0)
    tpr_mean[-1] = 1.0
    tpr_std      = 2*np.std(interp_tprs, axis=0)
    tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
    tpr_lower    = tpr_mean-tpr_std
    auc          = np.mean(results[kind]['auc'])
    
    roc_dict = {}
    roc_dict['tpr_mean'] = tpr_mean
    roc_dict['tpr_std'] = tpr_std
    roc_dict['tpr_upper'] = tpr_upper
    roc_dict['tpr_lower'] = tpr_lower
    roc_dict['auc'] = auc

    if(fig_show):
        fig = go.Figure([
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_upper,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'upper'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_lower,
                fill       = 'tonexty',
                fillcolor  = c_fill,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'lower'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_mean,
                line       = dict(color=c_line_main, width=2),
                hoverinfo  = "skip",
                showlegend = True,
                name       = f'AUC: {auc:.3f}')
        ])

        fig.add_shape(
            type ='line', 
            line =dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
        )

        fig.update_layout(
            template    = 'plotly_white', 
            title_x     = 0.5,
            xaxis_title = "1 - Specificity",
            yaxis_title = "Sensitivity",
            width       = 800,
            height      = 800,
            legend      = dict(
                yanchor="bottom", 
                xanchor="right", 
                x=0.95,
                y=0.01,
            )
        )

        fig.update_yaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            scaleanchor = "x", 
            scaleratio  = 1,
            linecolor   = 'black')

        fig.update_xaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            constrain   = 'domain',
            linecolor   = 'black')



        fig.show()
        
    else:
        print(y.name, 'AUC: ', np.round(auc,2), '+-( ', np.round(np.std(results[kind]['auc']),2), ')', X_.shape)

    return (model, results, np.round(auc,2), roc_dict)

In [9]:
from skopt import BayesSearchCV
def SearchBestModel(case_x, case_y):

    X_train, X_test, y_train, y_test = train_test_split(case_x, case_y, test_size=.2, random_state=1)

    opt_XGB = BayesSearchCV(
            XGBClassifier(),
            {
                'objective'   : ['binary:logistic'],
                'eval_metric' : ['logloss', 'error', 'auc', 'aucpr'],
                'learning_rate': (0.01, 0.1, 0.2),
                'max_depth': (3, 4, 5),
                'subsample': (0.8, 0.9, 1.0),
                'colsample_bytree': (0.8, 0.9, 1.0),
                'reg_alpha': (0, 0.1, 1.0),
                'reg_lambda': (0, 0.1, 1.0),
                #'gamma': (0.001, 0.01, 0.1, 1, 10),
            },
                          
            #n_iter=10,
            #cv=3,
            random_state=1
        )

    opt_XGB.fit(X_train, y_train)


    print('XGB')
    print("val. score: %s" % opt_XGB.best_score_)
    print("test score: %s" % opt_XGB.score(X_test, y_test))
    print("best parameters: %s" % str(opt_XGB.best_params_))
    print('---------------------------------------------\n')
    
    return opt_XGB

## Statistical significance check

In [10]:
compl_df.columns

Index(['SSV_IFG_L', 'SSV_IFG_R', 'SSV_TEMP_left', 'SSV_TEMP_right',
       'SSV_OCC_left', 'SSV_OCC_right', 'WMEAN_ALL', 'PE_ALL', 'FD_ALL', 'Age',
       'Sex', 'Educ', 'MMSE', 'GINI', 'ODQ',
       'max_value_lt_threshold_SSV_IFG_L', 'max_value_gt_threshold_SSV_IFG_L',
       'max_value_lt_threshold_SSV_IFG_R', 'max_value_gt_threshold_SSV_IFG_R',
       'max_value_lt_threshold_SSV_TEMP_left',
       'max_value_gt_threshold_SSV_TEMP_left',
       'max_value_lt_threshold_SSV_TEMP_right',
       'max_value_gt_threshold_SSV_TEMP_right',
       'max_value_lt_threshold_SSV_OCC_left',
       'max_value_gt_threshold_SSV_OCC_left',
       'max_value_lt_threshold_SSV_OCC_right',
       'max_value_gt_threshold_SSV_OCC_right',
       'max_value_lt_threshold_WMEAN_ALL', 'max_value_gt_threshold_WMEAN_ALL',
       'max_value_lt_threshold_PE_ALL', 'max_value_gt_threshold_PE_ALL',
       'max_value_lt_threshold_FD_ALL', 'max_value_gt_threshold_FD_ALL',
       'SSV_IFG_L_normalized', 'SSV_IFG_R_normal

In [11]:
outcomes_list = list(compl_df.iloc[:,-9::])
df_metrics_slopes = pd.DataFrame(columns=outcomes_list)
df_metrics_pval = pd.DataFrame(columns=outcomes_list)
df_metrics_dir = pd.DataFrame(columns=outcomes_list)


In [12]:
outcomes_list

['SSV_IFG_L_normalized',
 'SSV_IFG_R_normalized',
 'SSV_TEMP_left_normalized',
 'SSV_TEMP_right_normalized',
 'SSV_OCC_left_normalized',
 'SSV_OCC_right_normalized',
 'WMEAN_ALL_normalized',
 'PE_ALL_normalized',
 'FD_ALL_normalized']

In [13]:
from scipy.stats import linregress

df_metrics_slopes = pd.DataFrame(columns=outcomes_list)

for variable_y in outcomes_list:
    
    c_compl_df_mmse = compl_df[['GINI', variable_y]]
    
    c_compl_df_mmse = c_compl_df_mmse.dropna()
    c_compl_df_mmse.reset_index(drop=True)
    
    y_compl_1A = c_compl_df_mmse[variable_y]
    X_compl_1A = c_compl_df_mmse[['GINI']]
    
    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)

    X_compl_1A =  X_compl_1A.copy()

    slope, intercept, r_value, p_value, std_err = linregress(X_compl_1A['GINI'], y_compl_1A)
    
    p_value*=df_metrics_slopes.shape[1]
    if(p_value<=0.05):
        df_metrics_slopes.loc['GINI', variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' 
    
    df_metrics_pval.loc['GINI', variable_y] =  p_value
    df_metrics_dir.loc['GINI', variable_y] =  slope
    
for variable_y in outcomes_list:
    for variable_x in ['Age', 'Educ', 'Sex', 'MMSE']:
    
        c_compl_df_mmse = compl_df[['GINI', variable_x, variable_y]] 
        c_compl_df_mmse = c_compl_df_mmse.dropna()
        c_compl_df_mmse.reset_index(drop=True)
        
        y_compl_1A = c_compl_df_mmse[variable_y]
        X_compl_1A = c_compl_df_mmse[['GINI', variable_x]]
        
        scaler = MinMaxScaler([0.05, 0.95])
        scaling_data = scaler.fit_transform(X_compl_1A)
        X_compl_1A = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)

        X_compl_1A =  X_compl_1A.copy()

        slope, intercept, r_value, p_value, std_err = linregress(X_compl_1A[variable_x],y_compl_1A)
    
        p_value*=df_metrics_slopes.shape[1]
        if(p_value<=0.05):
            df_metrics_slopes.loc[variable_x, variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' #slope
     
        df_metrics_pval.loc[variable_x, variable_y] =  p_value
        df_metrics_dir.loc[variable_x, variable_y] =  slope

In [14]:
df_metrics_slopes = df_metrics_slopes.T
df_metrics_pval = df_metrics_pval.T
df_metrics_dir = df_metrics_dir.T

In [15]:
df_metrics_slopes

,GINI,Age,Sex,MMSE
SSV_IFG_L_normalized,-0.277 (0.0),0.387 (0.0),NaN,NaN
SSV_IFG_R_normalized,-0.244 (0.0),0.331 (0.0),NaN,NaN
SSV_TEMP_left_normalized,-0.22 (0.0),0.399 (0.0),NaN,NaN
SSV_TEMP_right_normalized,-0.274 (0.0),0.372 (0.0),NaN,NaN
SSV_OCC_left_normalized,-0.283 (0.0),0.412 (0.0),NaN,NaN
SSV_OCC_right_normalized,-0.229 (0.0),0.355 (0.0),NaN,NaN
WMEAN_ALL_normalized,-0.381 (0.0),0.156 (0.0),-0.059 (0.001),0.182 (0.001)
PE_ALL_normalized,-0.487 (0.0),0.226 (0.0),NaN,0.204 (0.025)
FD_ALL_normalized,-0.249 (0.0),NaN,-0.065 (0.0),0.164 (0.0)


In [16]:
df_metrics_GINI = df_metrics_slopes[['GINI']]
df_metrics_GINI = df_metrics_GINI.copy()
df_metrics_GINI.dropna(inplace=True)

In [17]:
outcomes_list = list(df_metrics_GINI.index)


## Classification

In [18]:
gini_results_preliminar = {}
gini_results_auc_preliminar = {}
for outcome in outcomes_list:

    compl_df_mmse = compl_df[[outcome, 'GINI']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

    mediana = compl_df[outcome].median()
    compl_df_mmse[outcome + '_b'] = (compl_df[outcome] > mediana).astype(int)

    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI']]

    scaler = MinMaxScaler((0.05, 0.95))
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()
    
    try:
        r = classifier_ROC(X_compl_1A, y_compl_1A, False)
        gini_results_preliminar['_'.join(outcome.split('_')[0:-1])] = r
        gini_results_auc_preliminar['_'.join(outcome.split('_')[0:-1])] = r[2]
    except:
        print('Bad: ',outcome + '_b')
        

  0%|          | 0/20 [00:00<?, ?it/s]

SSV_IFG_L_normalized_b AUC:  0.93 +-(  0.0 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

SSV_IFG_R_normalized_b AUC:  0.92 +-(  0.0 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

...

WMEAN_ALL_normalized_b AUC:  0.99 +-(  0.0 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

PE_ALL_normalized_b AUC:  0.99 +-(  0.0 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

FD_ALL_normalized_b AUC:  0.94 +-(  0.0 ) (1376, 1)


In [19]:
top_10_keys_GINI_preliminar = heapq.nlargest(10, gini_results_auc_preliminar, key=gini_results_auc_preliminar.get)
top_10_keys_GINI_preliminar

['WMEAN_ALL',
 'PE_ALL',
 'FD_ALL',
 'SSV_IFG_L',
 'SSV_OCC_left',
 'SSV_IFG_R',
 'SSV_TEMP_left',
 'SSV_TEMP_right',
 'SSV_OCC_right']

In [20]:
gini_results = {}
gini_results_auc = {}
gini_paragram = {}
for outcome in top_10_keys_GINI_preliminar:

    compl_df_mmse = compl_df[[outcome + '_normalized', 'GINI']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

    mediana = compl_df[outcome + '_normalized'].median()
    compl_df_mmse[outcome + '_b'] = (compl_df[outcome + '_normalized'] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI']]

    scaler = MinMaxScaler((0.05, 0.95))
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_compl_1A, y_compl_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_compl_1A, y_compl_1A, False, params_)
    gini_results[outcome] = r
    gini_results_auc[outcome] = r[2]
    gini_paragram[outcome] = params_


XGB
val. score: 0.9281818181818181
test score: 0.9420289855072463
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'auc'), ('learning_rate', 0.2), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.1), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

WMEAN_ALL_b AUC:  0.98 +-(  0.0 ) (1376, 1)
XGB
val. score: 0.9509090909090909
test score: 0.9492753623188406
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'aucpr'), ('learning_rate', 0.01), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 0.0), ('reg_lambda', 0.1), ('subsample', 1.0)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

PE_ALL_b AUC:  0.99 +-(  0.0 ) (1376, 1)
XGB
val. score: 0.8927272727272726
test score: 0.9021739130434783
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'auc'), ('learning_rate', 0.01), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.0), ('subsample', 1.0)])
---------------------------------------------



...

SSV_TEMP_left_b AUC:  0.92 +-(  0.0 ) (1376, 1)
XGB
val. score: 0.8727272727272727
test score: 0.8514492753623188
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'aucpr'), ('learning_rate', 0.01), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.0), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

SSV_TEMP_right_b AUC:  0.89 +-(  0.0 ) (1376, 1)
XGB
val. score: 0.86
test score: 0.8659420289855072
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'auc'), ('learning_rate', 0.1), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 0.0), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

SSV_OCC_right_b AUC:  0.9 +-(  0.0 ) (1376, 1)


In [21]:
top_10_keys_GINI = heapq.nlargest(10, gini_results_auc, key=gini_results_auc.get)
top_10_keys_GINI
gini_results_auc

{'WMEAN_ALL': 0.98,
 'PE_ALL': 0.99,
 'FD_ALL': 0.94,
 'SSV_IFG_L': 0.93,
 'SSV_OCC_left': 0.94,
 'SSV_IFG_R': 0.91,
 'SSV_TEMP_left': 0.92,
 'SSV_TEMP_right': 0.89,
 'SSV_OCC_right': 0.9}

## Classification: Gini + Age

In [22]:
gini_age_results_preliminar = {}
gini_age_results_auc_preliminar = {}
for outcome in outcomes_list:

    compl_df_mmse = compl_df[[outcome, 'GINI', 'Age']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()
    
    try:
        r = classifier_ROC(X_compl_1A, y_compl_1A, False)
        gini_age_results_preliminar['_'.join(outcome.split('_')[0:-1])] = r
        gini_age_results_auc_preliminar['_'.join(outcome.split('_')[0:-1])] = r[2]
    except:
        print('Bad: ',outcome + '_b')
        

  0%|          | 0/20 [00:00<?, ?it/s]

SSV_IFG_L_normalized_b AUC:  0.92 +-(  0.0 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

SSV_IFG_R_normalized_b AUC:  0.91 +-(  0.0 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

...

WMEAN_ALL_normalized_b AUC:  0.98 +-(  0.0 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

PE_ALL_normalized_b AUC:  0.99 +-(  0.0 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

FD_ALL_normalized_b AUC:  0.93 +-(  0.0 ) (1376, 2)


In [23]:
top_10_keys_GINI_AGE_preliminar = heapq.nlargest(10, gini_age_results_auc_preliminar, key=gini_age_results_auc_preliminar.get)

In [ ]:
gini_age_results = {}
gini_age_results_auc = {}
gini_age_paragram = {}
for outcome in top_10_keys_GINI_AGE_preliminar:

    compl_df_mmse = compl_df[[outcome + '_normalized', 'GINI', 'Age']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome + '_normalized'].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome + '_normalized'] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_compl_1A, y_compl_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_compl_1A, y_compl_1A, False, params_)
    gini_age_results[outcome] = r
    gini_age_results_auc[outcome] = r[2]
    gini_age_paragram[outcome] = params_


XGB
val. score: 0.9445454545454546
test score: 0.9492753623188406
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'error'), ('learning_rate', 0.1), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 1.0), ('subsample', 1.0)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

PE_ALL_b AUC:  0.99 +-(  0.0 ) (1376, 2)
XGB
val. score: 0.9290909090909093
test score: 0.9456521739130435
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'error'), ('learning_rate', 0.01), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 0.0), ('reg_lambda', 1.0), ('subsample', 0.8)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

WMEAN_ALL_b AUC:  0.98 +-(  0.0 ) (1376, 2)
XGB
val. score: 0.8918181818181818
test score: 0.894927536231884
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'error'), ('learning_rate', 0.1), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 0.0), ('reg_lambda', 0.0), ('subsample', 0.8)])
---------------------------------------------



...

SSV_IFG_R_b AUC:  0.9 +-(  0.01 ) (1376, 2)
XGB
val. score: 0.8699999999999999
test score: 0.8586956521739131
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'error'), ('learning_rate', 0.01), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 0.0), ('reg_lambda', 0.0), ('subsample', 0.8)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

SSV_TEMP_left_b AUC:  0.91 +-(  0.0 ) (1376, 2)
XGB
val. score: 0.8736363636363637
test score: 0.8586956521739131
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'auc'), ('learning_rate', 0.1), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 1.0), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

SSV_TEMP_right_b AUC:  0.88 +-(  0.01 ) (1376, 2)


In [ ]:
top_10_keys_GINI_age = heapq.nlargest(10, gini_age_results_auc, key=gini_age_results_auc.get)
top_10_keys_GINI_age
gini_results_auc
gini_age_results_auc

## Classification: Gini + Educ

In [ ]:
gini_educ_results_preliminar = {}
gini_educ_results_auc_preliminar = {}
for outcome in outcomes_list:

    compl_df_mmse = compl_df[[outcome, 'GINI', 'Educ']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()
    
    try:
        r = classifier_ROC(X_compl_1A, y_compl_1A, False)
        gini_educ_results_preliminar['_'.join(outcome.split('_')[0:-1])] = r
        gini_educ_results_auc_preliminar['_'.join(outcome.split('_')[0:-1])] = r[2]
    except:
        print('Bad: ',outcome + '_b')
        

In [ ]:
top_10_keys_GINI_educ_preliminar = heapq.nlargest(10, gini_educ_results_auc_preliminar, key=gini_educ_results_auc_preliminar.get)

In [ ]:
gini_educ_results = {}
gini_educ_results_auc = {}
gini_educ_paragram = {}
for outcome in top_10_keys_GINI_educ_preliminar:

    compl_df_mmse = compl_df[[outcome + '_normalized', 'GINI', 'Educ']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome + '_normalized'].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome + '_normalized'] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_compl_1A, y_compl_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_compl_1A, y_compl_1A, False, params_)
    gini_educ_results[outcome] = r
    gini_educ_results_auc[outcome] = r[2]
    gini_educ_paragram[outcome] = params_

In [ ]:
top_10_keys_gini_educ = heapq.nlargest(10, gini_educ_results_auc, key=gini_educ_results_auc.get)
top_10_keys_gini_educ
top_10_keys_gini_educ
gini_educ_results_auc



## Classification: Gini + Sex

In [ ]:
gini_sex_results_preliminar = {}
gini_sex_results_auc_preliminar = {}
for outcome in outcomes_list:

    compl_df_mmse = compl_df[[outcome, 'GINI', 'Sex']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()
    
    try:
        r = classifier_ROC(X_compl_1A, y_compl_1A, False)
        gini_sex_results_preliminar['_'.join(outcome.split('_')[0:-1])] = r
        gini_sex_results_auc_preliminar['_'.join(outcome.split('_')[0:-1])] = r[2]
    except:
        print('Bad: ',outcome + '_b')
        

In [ ]:
top_10_keys_GINI_sex_preliminar = heapq.nlargest(10, gini_sex_results_auc_preliminar, key=gini_sex_results_auc_preliminar.get)
   

In [ ]:
     
gini_sex_results = {}
gini_sex_results_auc = {}
gini_sex_paragram = {}
for outcome in top_10_keys_GINI_sex_preliminar:

    compl_df_mmse = compl_df[[outcome + '_normalized', 'GINI', 'Sex']]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome + '_normalized'].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome + '_normalized'] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_compl_1A, y_compl_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_compl_1A, y_compl_1A, False, params_)
    gini_sex_results[outcome] = r
    gini_sex_results_auc[outcome] = r[2]
    gini_sex_paragram[outcome] = params_

In [ ]:

top_10_keys_gini_sex = heapq.nlargest(10, gini_sex_results_auc, key=gini_sex_results_auc.get)
top_10_keys_gini_sex
top_10_keys_gini_educ
gini_sex_results_auc



## Classification: Gini + Cognition

In [ ]:
gini_mmse_results_preliminar = {}
gini_mmse_results_auc_preliminar = {}
for outcome in outcomes_list:

    compl_df_mmse = compl_df[[outcome, 'GINI', 'MMSE']]
    compl_df_mmse = compl_df_mmse[compl_df_mmse['MMSE'] >= 21]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()
    
    try:
        r = classifier_ROC(X_compl_1A, y_compl_1A, False)
        gini_mmse_results_preliminar['_'.join(outcome.split('_')[0:-1])] = r
        gini_mmse_results_auc_preliminar['_'.join(outcome.split('_')[0:-1])] = r[2]
    except:
        print('Bad: ',outcome + '_b')
        
        

In [ ]:
top_10_keys_GINI_mmse_preliminar = heapq.nlargest(10, gini_mmse_results_auc_preliminar, key=gini_mmse_results_auc_preliminar.get)

In [ ]:
gini_mmse_results = {}
gini_mmse_results_auc = {}
gini_mmse_paragram = {}
for outcome in top_10_keys_GINI_mmse_preliminar:

    compl_df_mmse = compl_df[[outcome + '_normalized', 'GINI', 'MMSE']]
    compl_df_mmse = compl_df_mmse[compl_df_mmse['MMSE'] >= 21]
    compl_df_mmse = compl_df_mmse.dropna()
    compl_df_mmse.reset_index(drop=True)

   
    mediana = compl_df_mmse[outcome + '_normalized'].median()
    compl_df_mmse[outcome + '_b'] = (compl_df_mmse[outcome + '_normalized'] > mediana).astype(int)
    
    y_compl_1A = compl_df_mmse[outcome + '_b']
    X_compl_1A = compl_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_compl_1A, y_compl_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_compl_1A, y_compl_1A, False, params_)
    gini_mmse_results[outcome] = r
    gini_mmse_results_auc[outcome] = r[2]
    gini_mmse_paragram[outcome] = params_


In [ ]:
top_10_keys_gini_mmse = heapq.nlargest(10, gini_mmse_results_auc, key=gini_mmse_results_auc.get)
top_10_keys_gini_mmse
gini_sex_results_auc
gini_mmse_results_auc